In [1]:
! pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
produtos = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/produtos.csv', header=True, inferSchema=True)
vendedores = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/vendedores.csv', header=True, inferSchema=True)
clientes = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/itens_pedido.csv', header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pagamentos_pedido.csv', header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/avaliacoes_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pedidos.csv', header=True, inferSchema=True)

In [5]:
produtos.show(5)

+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|1e9e8ef04dbcff454...|          perfumaria|                  40|                      287|                       1|           225|                    16|               10|                14|
|3aa071139cb16b67c...|               artes|                  44|                      276|                       1|          1000|                    30|               18|                20|
|96bd76ec8810374ed...|       esporte_lazer|  

In [6]:
qtd_produto = produtos.groupBy('categoria_produto').agg(
    count_distinct('id_produto').alias('contagem_unicos_exata'),
    approx_count_distinct('id_produto').alias('contagem_unicos_aproximada'))

qtd_produto.show()

+--------------------+---------------------+--------------------------+
|   categoria_produto|contagem_unicos_exata|contagem_unicos_aproximada|
+--------------------+---------------------+--------------------------+
|                 pcs|                   30|                        30|
|               bebes|                  919|                       873|
|               artes|                   55|                        56|
|           cine_foto|                   28|                        29|
|    moveis_decoracao|                 2657|                      2715|
|            pc_gamer|                    3|                         3|
|tablets_impressao...|                    9|                         9|
|construcao_ferram...|                  400|                       401|
|fashion_roupa_mas...|                   95|                        98|
|   artigos_de_festas|                   26|                        26|
|    artigos_de_natal|                   65|                    

In [8]:
pedidos_completos_df = pedidos.dropna()

In [12]:
pedidos_completos_df.show(5)

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|    delivered|2017-10-02 10:56:33|  2017-10-02 11:07:15|      2017-10-04 19:55:00| 2017-10-10 21:25:13|  2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|    delivered|2018-07-24 20:41:37|  2018-07-26 03:24:27|      2018-07-26 14:31:00| 2018-08-07 15:27:45|  2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|    delivered|2018-08-08 08:38:49|  2018-08-08 08:55:23|      2018-08-08 13:50:00| 2018-08-17 18:06:29|  2018-09

In [13]:
pedidos_completos_df = pedidos_completos_df.withColumn(
    'tempo_envio_dias',
    (col('data_entrega_cliente').cast('long') - col('data_aprovacao_pedido').cast('long'))/86400)

pedidos_completos_df.show(5)

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|  tempo_envio_dias|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|    delivered|2017-10-02 10:56:33|  2017-10-02 11:07:15|      2017-10-04 19:55:00| 2017-10-10 21:25:13|  2017-10-18 00:00:00| 8.429143518518519|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|    delivered|2018-07-24 20:41:37|  2018-07-26 03:24:27|      2018-07-26 14:31:00| 2018-08-07 15:27:45|  2018-08-13 00:00:00|12.502291666666666|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|    delivered|201

In [18]:
janela_acumulativa = Window.partitionBy('status_pedido').orderBy('data_aprovacao_pedido').rowsBetween(Window.unboundedPreceding, Window.currentRow)

pedidos_completos_df = pedidos_completos_df.withColumn(
    'media_tempo_envio_cumulativa',
    avg('tempo_envio_dias').over(janela_acumulativa))

pedidos_completos_df.show(20)

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+----------------------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|  tempo_envio_dias|media_tempo_envio_cumulativa|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+----------------------------+
|65d1e226dfaeb8cdc...|70fc57eeae2926759...|     canceled|2016-10-03 21:01:41|  2016-10-04 10:18:57|      2016-10-25 12:14:28| 2016-11-08 10:58:34|  2016-11-25 00:00:00|35.027511574074076|          35.027511574074076|
|770d331c84e5b214b...|6c57e6119369185e5...|     canceled|2016-10-07 14:52:30|  2016-10-07 15:07:10|      2016-10-11 15:07:11| 2016-1

In [19]:
janela_10dias = Window.partitionBy('status_pedido').orderBy('data_aprovacao_pedido').rowsBetween(-9, 0)

pedidos_completos_df = pedidos_completos_df.withColumn(
    'media_tempo_envio_cumulativa',
    avg('tempo_envio_dias').over(janela_10dias))

pedidos_completos_df.show(20)

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+----------------------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|  tempo_envio_dias|media_tempo_envio_cumulativa|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+------------------+----------------------------+
|65d1e226dfaeb8cdc...|70fc57eeae2926759...|     canceled|2016-10-03 21:01:41|  2016-10-04 10:18:57|      2016-10-25 12:14:28| 2016-11-08 10:58:34|  2016-11-25 00:00:00|35.027511574074076|          35.027511574074076|
|770d331c84e5b214b...|6c57e6119369185e5...|     canceled|2016-10-07 14:52:30|  2016-10-07 15:07:10|      2016-10-11 15:07:11| 2016-1

In [20]:
windowSpec = Window.partitionBy('status_pedido').orderBy('data_aprovacao_pedido').rowsBetween(Window.unboundedPreceding, Window.currentRow)

pedidos_ordenados = pedidos.orderBy('data_aprovacao_pedido')
pedidos_ordenados = pedidos_ordenados.dropna()

print(pedidos.count())

tempo_medio_cumulativo = pedidos_ordenados.withColumn(
    'tempo_envio_dias',
    (col('data_entrega_cliente').cast('long') - col('data_aprovacao_pedido').cast('long')) / 86400
).withColumn(
    'media_tempo_envio_cumulativa',
    avg('tempo_envio_dias').over(windowSpec)
)

tempo_medio_cumulativo.select('status_pedido', 'data_aprovacao_pedido', 'media_tempo_envio_cumulativa').show()

99441
+-------------+---------------------+----------------------------+
|status_pedido|data_aprovacao_pedido|media_tempo_envio_cumulativa|
+-------------+---------------------+----------------------------+
|     canceled|  2016-10-04 10:18:57|          35.027511574074076|
|     canceled|  2016-10-07 15:07:10|          21.013761574074074|
|     canceled|  2016-10-09 13:36:58|          16.356400462962963|
|     canceled|  2016-10-09 14:34:30|          14.811261574074074|
|     canceled|  2016-10-10 10:40:49|          17.884150462962964|
|     canceled|  2018-02-19 20:56:05|          19.911302083333336|
|    delivered|  2016-09-15 12:16:38|           54.81319444444444|
|    delivered|  2016-10-04 09:43:32|           37.54903356481481|
|    delivered|  2016-10-04 10:19:23|          28.446720679012344|
|    delivered|  2016-10-04 10:25:46|           32.56860821759259|
|    delivered|  2016-10-04 10:28:07|          31.460384259259257|
|    delivered|  2016-10-04 10:28:19|          30.916805

In [21]:
pedidos_com_mes = pedidos.withColumn('mes_compra', month('data_compra_pedido'))
vendas_por_mes_status = pedidos_com_mes.groupBy('mes_compra').pivot('status_pedido').agg(
    count('id_pedido').alias('total_pedidos')).orderBy('mes_compra')

vendas_por_mes_status.show()

+----------+--------+--------+-------+---------+--------+----------+-------+-----------+
|mes_compra|approved|canceled|created|delivered|invoiced|processing|shipped|unavailable|
+----------+--------+--------+-------+---------+--------+----------+-------+-----------+
|         1|    NULL|      37|   NULL|     7819|      27|        38|     90|         58|
|         2|       1|      90|      1|     8208|      17|        38|     78|         75|
|         3|    NULL|      59|   NULL|     9549|      26|        32|    178|         49|
|         4|       1|      33|   NULL|     9101|      28|        18|    148|         14|
|         5|    NULL|      53|   NULL|    10295|      40|        29|    109|         47|
|         6|    NULL|      34|   NULL|     9234|      14|        12|     90|         28|
|         7|    NULL|      69|   NULL|    10031|      20|        12|    116|         70|
|         8|    NULL|     111|   NULL|    10544|      43|        18|     88|         39|
|         9|    NULL|

In [22]:
spark.stop()